# Final Project for the Applied Data Science Course at Reykjavik University
### This notebook uses the [Apartment rental offers in Germany](https://www.kaggle.com/datasets/corrieaar/apartment-rental-offers-in-germany?resource=download) dataset from kaggle
### Alice Tedeschi & Mischa Rauch

In [2]:
# Imports
import tensorflow as tf
import pandas as pd
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt


# Optionally run this cell to get interactive (zoom-able and pan-able)
# matplotlib figures. This does not work e.g. in Google Colab.
%matplotlib notebook
# Widen the output format for our dataset which contains at most 49 columns
pd.set_option('display.max_columns', 49)

2022-12-02 10:26:18.060597: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


### Data exploration

In [5]:
# Load Data
df = pd.read_csv('data/immo_data.csv')

In [8]:
df.shape

(268850, 49)

In [7]:
df.describe(include='all')

,regio1,serviceCharge,heatingType,telekomTvOffer,telekomHybridUploadSpeed,newlyConst,balcony,picturecount,pricetrend,telekomUploadSpeed,...,regio2,regio3,description,facilities,heatingCosts,energyEfficiencyClass,lastRefurbish,electricityBasePrice,electricityKwhPrice,date
count,268850,261941.000000,223994,236231,45020.0,268850,268850,268850.000000,267018.000000,235492.000000,...,268850,268850,249103,215926,85518.000000,77787,80711.000000,46846.000000,46846.000000,268850
unique,16,NaN,13,3,NaN,2,2,NaN,NaN,NaN,...,419,8684,212621,189526,NaN,10,NaN,NaN,NaN,4
top,Nordrhein_Westfalen,NaN,central_heating,ONE_YEAR_FREE,NaN,False,True,NaN,NaN,NaN,...,Leipzig,Innenstadt,+ MEIN GUENTZ - Quartier der Generationen\r\n+...,Laminat,NaN,C,NaN,NaN,NaN,Feb20
freq,62863,NaN,128977,227632,NaN,247679,165734,NaN,NaN,NaN,...,13723,4751,216,207,NaN,14613,NaN,NaN,NaN,79276
mean,NaN,151.206113,NaN,NaN,10.0,NaN,NaN,9.791958,3.389001,28.804928,...,NaN,NaN,NaN,NaN,76.990866,NaN,2013.904536,89.113612,0.199769,NaN
std,NaN,308.295790,NaN,NaN,0.0,NaN,NaN,6.408399,1.964874,16.337151,...,NaN,NaN,NaN,NaN,147.716278,NaN,10.963125,5.395805,0.009667,NaN
min,NaN,0.000000,NaN,NaN,10.0,NaN,NaN,0.000000,-12.330000,1.000000,...,NaN,NaN,NaN,NaN,0.000000,NaN,1015.000000,71.430000,0.170500,NaN
25%,NaN,95.000000,NaN,NaN,10.0,NaN,NaN,6.000000,2.000000,10.000000,...,NaN,NaN,NaN,NaN,54.000000,NaN,2012.000000,90.760000,0.191500,NaN
50%,NaN,135.000000,NaN,NaN,10.0,NaN,NaN,9.000000,3.390000,40.000000,...,NaN,NaN,NaN,NaN,70.000000,NaN,2017.000000,90.760000,0.198500,NaN
75%,NaN,190.000000,NaN,NaN,10.0,NaN,NaN,13.000000,4.570000,40.000000,...,NaN,NaN,NaN,NaN,90.000000,NaN,2019.000000,90.760000,0.205500,NaN


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 268850 entries, 0 to 268849
Data columns (total 49 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   regio1                    268850 non-null  object 
 1   serviceCharge             261941 non-null  float64
 2   heatingType               223994 non-null  object 
 3   telekomTvOffer            236231 non-null  object 
 4   telekomHybridUploadSpeed  45020 non-null   float64
 5   newlyConst                268850 non-null  bool   
 6   balcony                   268850 non-null  bool   
 7   picturecount              268850 non-null  int64  
 8   pricetrend                267018 non-null  float64
 9   telekomUploadSpeed        235492 non-null  float64
 10  totalRent                 228333 non-null  float64
 11  yearConstructed           211805 non-null  float64
 12  scoutId                   268850 non-null  int64  
 13  noParkSpaces              93052 non-null   f

In [15]:
df.head()

,regio1,serviceCharge,heatingType,telekomTvOffer,telekomHybridUploadSpeed,newlyConst,balcony,picturecount,pricetrend,telekomUploadSpeed,...,regio2,regio3,description,facilities,heatingCosts,energyEfficiencyClass,lastRefurbish,electricityBasePrice,electricityKwhPrice,date
0,Nordrhein_Westfalen,245.00,central_heating,ONE_YEAR_FREE,NaN,False,False,6,4.62,10.0,...,Dortmund,Schüren,Die ebenerdig zu erreichende Erdgeschosswohnun...,Die Wohnung ist mit Laminat ausgelegt. Das Bad...,NaN,NaN,NaN,NaN,NaN,May19
1,Rheinland_Pfalz,134.00,self_contained_central_heating,ONE_YEAR_FREE,NaN,False,True,8,3.47,10.0,...,Rhein_Pfalz_Kreis,Böhl_Iggelheim,Alles neu macht der Mai – so kann es auch für ...,NaN,NaN,NaN,2019.0,NaN,NaN,May19
2,Sachsen,255.00,floor_heating,ONE_YEAR_FREE,10.0,True,True,8,2.72,2.4,...,Dresden,Äußere_Neustadt_Antonstadt,Der Neubau entsteht im Herzen der Dresdner Neu...,"* 9 m² Balkon\n* Bad mit bodengleicher Dusche,...",NaN,NaN,NaN,NaN,NaN,Oct19
3,Sachsen,58.15,district_heating,ONE_YEAR_FREE,NaN,False,True,9,1.53,40.0,...,Mittelsachsen_Kreis,Freiberg,Abseits von Lärm und Abgasen in Ihre neue Wohn...,NaN,87.23,NaN,NaN,NaN,NaN,May19
4,Bremen,138.00,self_contained_central_heating,NaN,NaN,False,True,19,2.46,NaN,...,Bremen,Neu_Schwachhausen,Es handelt sich hier um ein saniertes Mehrfami...,Diese Wohnung wurde neu saniert und ist wie fo...,NaN,NaN,NaN,NaN,NaN,Feb20


In [35]:
df.isna().sum()

regio1                           0
serviceCharge                 6909
heatingType                  44856
telekomTvOffer               32619
telekomHybridUploadSpeed    223830
newlyConst                       0
balcony                          0
picturecount                     0
pricetrend                    1832
telekomUploadSpeed           33358
totalRent                    40517
yearConstructed              57045
scoutId                          0
noParkSpaces                175798
firingTypes                  56964
hasKitchen                       0
geo_bln                          0
cellar                           0
yearConstructedRange         57045
baseRent                         0
houseNumber                  71018
livingSpace                      0
geo_krs                          0
condition                    68489
interiorQual                112665
petsAllowed                 114573
street                           0
streetPlain                  71013
lift                

In [36]:
df.isna().sum() > df.shape[0]*0.75
# telekomHybridUploadSpeed, electricityBasePrice, electricityKwhPrice are useless

regio1                      False
serviceCharge               False
heatingType                 False
telekomTvOffer              False
telekomHybridUploadSpeed     True
newlyConst                  False
balcony                     False
picturecount                False
pricetrend                  False
telekomUploadSpeed          False
totalRent                   False
yearConstructed             False
scoutId                     False
noParkSpaces                False
firingTypes                 False
hasKitchen                  False
geo_bln                     False
cellar                      False
yearConstructedRange        False
baseRent                    False
houseNumber                 False
livingSpace                 False
geo_krs                     False
condition                   False
interiorQual                False
petsAllowed                 False
street                      False
streetPlain                 False
lift                        False
baseRentRange 

In [53]:
print(df['description'].unique().shape[0]/df.shape[0])
print(df['facilities'].unique().shape[0]/df.shape[0])
# ~80% of the descriptions and 70% of the facilities descriptions are unique -> not very useful

0.7908573554026409
0.7049544355588618


In [62]:
df.value_counts('regio1') # == df.value_counts('geo_bln') --> drop geo_bln

regio1
Nordrhein_Westfalen       62863
Sachsen                   58154
Bayern                    21609
Sachsen_Anhalt            20124
Hessen                    17845
Niedersachsen             16593
Baden_Württemberg         16091
Berlin                    10406
Thüringen                  8388
Rheinland_Pfalz            8368
Brandenburg                6954
Schleswig_Holstein         6668
Mecklenburg_Vorpommern     6634
Hamburg                    3759
Bremen                     2965
Saarland                   1429
dtype: int64

In [96]:
df.value_counts('regio2') # == df.value_counts('geo_krs') --> drop geo_krs

regio2
Leipzig                         13723
Chemnitz                        12575
Berlin                          10405
Dresden                          7522
Magdeburg                        4860
                                ...  
Lichtenfels_Kreis                  31
Freyung_Grafenau_Kreis             30
Haßberge_Kreis                     24
Neustadt_a.d._Waldnaab_Kreis       16
Kronach_Kreis                      12
Length: 419, dtype: int64

In [65]:
df.value_counts('heatingType')

heatingType
central_heating                   128977
district_heating                   24808
gas_heating                        19955
self_contained_central_heating     19087
floor_heating                      17697
oil_heating                         5042
heat_pump                           2737
combined_heat_and_power_plant       1978
night_storage_heater                1341
wood_pellet_heating                  961
electric_heating                     901
stove_heating                        344
solar_heating                        166
dtype: int64

In [66]:
df.value_counts('telekomTvOffer')

telekomTvOffer
ONE_YEAR_FREE    227632
NONE               4957
ON_DEMAND          3642
dtype: int64

In [13]:
df.value_counts(df['telekomHybridUploadSpeed']) # -> only value in dataset i.e. drop telekomHybridUploadSpeed 

telekomHybridUploadSpeed
10.0    45020
dtype: int64

In [68]:
df.value_counts(df['picturecount'])

picturecount
8      21930
7      21429
9      21421
10     19819
6      18848
       ...  
100        1
101        1
107        1
109        1
112        1
Length: 95, dtype: int64

In [69]:
df.value_counts(df['pricetrend'])

pricetrend
 0.00     11082
 3.33      2211
 3.23      2042
 3.85      1884
 3.57      1720
          ...  
-1.09         1
-0.98         1
-0.81         1
-0.73         1
 14.92        1
Length: 1234, dtype: int64

In [71]:
df.value_counts(df['telekomUploadSpeed'])

telekomUploadSpeed
40.0     158296
2.4       42858
10.0      32889
5.0        1036
1.0         209
100.0       141
4.0          63
dtype: int64

In [72]:
df.value_counts(df['totalRent']) # our y value

totalRent
500.00         1897
450.00         1892
600.00         1805
550.00         1668
400.00         1500
               ... 
586.38            1
586.36            1
586.33            1
586.32            1
15751535.00       1
Length: 28486, dtype: int64

In [91]:
print(df.value_counts(df['yearConstructed'])) # get rid of values higher than 2022
index_names = df[ df['yearConstructed'] > 2022 ].index
df.drop(index_names, inplace = True)

yearConstructed
2019.0    10959
1900.0    10356
2018.0     8759
1995.0     4387
1996.0     4339
          ...  
1683.0        1
1688.0        1
1693.0        1
1695.0        1
2090.0        1
Length: 465, dtype: int64


In [105]:
print(df.value_counts(df['noParkSpaces']))
df1 = df[df['noParkSpaces'] > 50]
print('Number of entries with more than 50 parking spaces... ',df1.shape[0])
index_names = df[ df['noParkSpaces'] > 50 ].index
df.drop(index_names, inplace = True)

noParkSpaces
1.0       78813
2.0        8955
0.0        3855
3.0         417
4.0         137
          ...  
73.0          1
64.0          1
59.0          1
58.0          1
2241.0        1
Length: 71, dtype: int64
Number of entries with more than 50 parking spaces...  89


In [108]:
df.value_counts(df['firingTypes'])

firingTypes
gas                                             110864
district_heating                                 49368
oil                                              18136
natural_gas_light                                10077
electricity                                       4838
                                                 ...  
pellet_heating:natural_gas_light                     1
gas:liquid_gas                                       1
pellet_heating:wood:wood_chips                       1
gas:electricity:environmental_thermal_energy         1
district_heating:natural_gas_heavy                   1
Length: 132, dtype: int64

In [130]:
print(df.value_counts(df['yearConstructed'])) # apparently 87 values were constructed in 1111 :D
index_names = df[ df['yearConstructed'] < 1500 ].index
df.drop(index_names, inplace = True)

yearConstructed
2019.0    10953
1900.0    10356
2018.0     8757
1995.0     4378
1996.0     4327
          ...  
1633.0        1
1634.0        1
1706.0        1
1645.0        1
1611.0        1
Length: 413, dtype: int64


In [138]:
print(df.value_counts(df['baseRent']))
# if the base rent is higher than 16.000 its a outlier
index_names = df[ df['baseRent'] > 16000 ].index
df.drop(index_names, inplace = True)

In [139]:
df.value_counts(df['houseNumber']) # we don't need the house number to predict prices

houseNumber
1        7168
2        6785
4        5820
3        5702
5        5510
         ... 
29B/C       1
29D         1
29g         1
2H          1
yy          1
Length: 5502, dtype: int64

In [146]:
print(df.value_counts(df['livingSpace'])) # in sqm
index_names = df[ df['livingSpace'] > 3000 ].index # 7 entries 
df.drop(index_names, inplace = True)

livingSpace
60.00        5188
70.00        4152
80.00        4030
65.00        3807
75.00        3690
             ... 
135.84          1
26.49           1
104.28          1
26.44           1
111111.00       1
Length: 12999, dtype: int64


In [147]:
df.value_counts(df['condition'])

condition
well_kept                             66557
refurbished                           26947
fully_renovated                       26344
first_time_use                        21944
mint_condition                        21903
modernized                            17213
first_time_use_after_refurbishment    15683
negotiable                             2238
need_of_renovation                     1371
ripe_for_demolition                       4
dtype: int64

In [148]:
df.value_counts(df['interiorQual'])

interiorQual
normal           81781
sophisticated    64705
luxury            7642
simple            1948
dtype: int64

In [149]:
df.value_counts(df['petsAllowed'])

petsAllowed
negotiable    91911
no            51953
yes           10312
dtype: int64

In [150]:
# keep streetPlain and drop street 

df.value_counts(df['baseRentRange'])

baseRentRange
2    54486
1    48183
5    37977
3    37104
7    27464
4    25785
6    24192
8     8242
9     5157
dtype: int64

In [151]:
df.value_counts(df['typeOfFlat'])

typeOfFlat
apartment              131382
roof_storey             34760
ground_floor            31506
other                    9510
maisonette               9314
raised_ground_floor      5625
penthouse                3563
terraced_flat            3384
half_basement            2013
loft                      954
dtype: int64

In [153]:
df.value_counts(df['geo_plz'])

geo_plz
9130     2008
9126     1993
9131     1649
9112     1626
9113     1416
         ... 
86707       1
54200       1
54309       1
54313       1
54492       1
Length: 7633, dtype: int64

In [158]:
print(df.value_counts(df['noRooms']))
index_names = df[ df['noRooms'] > 50 ].index # 14 entries
df.drop(index_names, inplace = True)

noRooms
3.00      92021
2.00      88958
4.00      27756
1.00      27750
2.50       9571
          ...  
2.65          1
2.70          1
2.90          1
3.40          1
999.99        1
Length: 63, dtype: int64


In [159]:
df.value_counts(df['energyEfficiencyClass']) # drop thermalChar since this is more insightful

thermalChar
105.00     932
100.00     913
114.00     876
80.00      871
87.00      863
          ... 
120.79       1
120.63       1
120.59       1
120.54       1
1996.00      1
Length: 7843, dtype: int64

In [171]:
print(df.value_counts(df['floor']))
index_names = df[ df['floor'] > 63 ].index # germanys highest building has 63 floors
df.drop(index_names, inplace = True)

floor
 1.0     64077
 2.0     56871
 3.0     37843
 0.0     24594
 4.0     19941
 5.0      7982
 6.0      2489
 7.0      1029
 8.0       577
 9.0       427
 10.0      363
-1.0       314
 11.0      236
 12.0      143
 13.0      133
 14.0      110
 15.0       61
 16.0       34
 17.0       27
 19.0       15
 21.0       12
 18.0       11
 20.0       10
 22.0        4
 24.0        3
 23.0        2
 26.0        2
 29.0        2
 36.0        1
 41.0        1
 37.0        1
 25.0        1
 32.0        1
 31.0        1
 45.0        1
dtype: int64


In [174]:
print(df.value_counts('numberOfFloors'))
index_names = df[ df['numberOfFloors'] > 63 ].index # germanys highest building has 63 floors
df.drop(index_names, inplace = True)

numberOfFloors
3.0     50779
4.0     39823
2.0     37338
5.0     19816
1.0      8458
6.0      6261
7.0      2550
0.0      1371
8.0      1133
11.0     1004
10.0      572
9.0       513
14.0      239
12.0      189
16.0      170
13.0      169
15.0      122
17.0      102
18.0       73
20.0       65
19.0       55
23.0       29
21.0       28
22.0       25
40.0       12
26.0        7
27.0        6
31.0        5
24.0        5
33.0        4
25.0        3
45.0        3
30.0        3
34.0        3
36.0        3
42.0        2
43.0        2
32.0        2
28.0        2
41.0        1
29.0        1
54.0        1
dtype: int64


In [175]:
df.value_counts(df['noRoomsRange'])

noRoomsRange
3    100910
2     98607
1     31150
4     29828
5      8034
dtype: int64

In [176]:
df.value_counts(df['livingSpaceRange'])

livingSpaceRange
3    82367
2    76294
4    42754
1    26999
5    20511
6    12857
7     6747
dtype: int64

In [181]:
print(df.value_counts(df['heatingCosts']))
index_names = df[ df['heatingCosts'] > 9000 ].index # 11 entries
df.drop(index_names, inplace = True)

heatingCosts
60.00       5302
70.00       4808
80.00       4756
50.00       4277
100.00      3965
            ... 
70.91          1
70.92          1
70.95          1
70.99          1
12613.00       1
Length: 5661, dtype: int64


In [192]:
print(df.value_counts(df['lastRefurbish']))
index_names = df[ df['lastRefurbish'] < 1100].index # 1 value 
df.drop(index_names, inplace = True)

lastRefurbish
2019.0    20180
2018.0    13191
2017.0     7094
2016.0     5669
2015.0     4632
          ...  
1957.0        1
1964.0        1
1965.0        1
1973.0        1
1015.0        1
Length: 88, dtype: int64


In [193]:
df.value_counts(df['electricityBasePrice']) # useless drop electricityBasePrice

electricityBasePrice
90.76    42813
71.43     3987
dtype: int64

In [195]:
df.value_counts(df['electricityKwhPrice'])

electricityKwhPrice
0.1985    14620
0.2055    13061
0.1915    10109
0.1845     2951
0.2276     1411
0.2125      726
0.2074      704
0.1775      675
0.2205      661
0.2137      450
0.2132      404
0.2144      357
0.1705      239
0.2195      239
0.2265      193
dtype: int64

In [196]:
df.value_counts(df['date'])

date
Feb20    79124
May19    75987
Oct19    66610
Sep18    46796
dtype: int64

In [ ]:
# drop scoutId -> immoscout id no information gain TODO: show no correlation 
# DROP: telekomHybridUploadSpeed, electricityBasePrice, electricityKwhPrice, description, facilities, scoutId, 
#       geo_bln, geo_krs, houseNumber, street, thermalChar, electricityBasePrice